# 📂 PJ01_Fintech_Data_Pipeline

### : 지저분한 주식 거래 로그 정제 및 가중평균(VWAP) 산출 시스템

## 1. 💼 가상 실무 시나리오 (Scenario)

**"고객님 계좌 잔고가 왜 이러죠? 시스템 오류인가요?"**

당신은 핀테크 스타트업 **'QuantLab(퀀트랩)'**의 주니어 데이터 엔지니어입니다.
오늘 아침, 제휴 증권사로부터 어제 자 **고객 주식 거래 내역(Raw Data)** 파일이 도착했습니다.

하지만 파일을 열어본 순간, 당신은 경악을 금치 못합니다.
레거시 시스템의 오류로 인해 데이터가 엉망진창으로 섞여 있었기 때문입니다.

* **날짜**는 `2024-01-01`, `2024/01/02`가 섞여 있고, 심지어 `invalid_date` 같은 문자열도 보입니다.
* **매수/매도 구분**은 `Buy`, `B`, `sell`, `S` 등 제각각이라 시스템이 인식을 못 합니다.
* **체결 가격** 중간중간이 비어있거나(`NaN`), 쉼표(`70,000`)가 붙어 있어 계산이 불가능합니다.

**팀장님의 긴급 요청:**

> "이 데이터 그대로 자산 평가 엔진에 넣으면 우리 서비스 셧다운 됩니다.
> **2시간 내**에 데이터를 완벽하게 **전처리(Cleaning)** 하고,
> 구멍 난 가격은 **해당 종목의 평균가**로 메꿔서,
> 종목별 **정확한 평단가(VWAP)**를 뽑아주세요."

당신의 임무는 이 '쓰레기 데이터'를 '황금 같은 정보'로 바꾸는 **데이터 파이프라인**을 구축하는 것입니다.

---

## 2. 📋 프로젝트 기획안 (Planning)

### **1) 프로젝트 목표**

1. **데이터 정규화 (Standardization):** 이질적인 날짜, 문자열 포맷을 통일하여 분석 가능한 상태로 만듭니다.
2. **결측치 보간 (Imputation):** 단순히 0이나 전체 평균을 넣는 것이 아니라, **'종목별 평균'**이라는 도메인 맥락에 맞는 값으로 결측치를 채웁니다.
3. **금융 지표 산출 (Financial Metrics):** 단순 평균이 아닌, 거래량을 고려한 **가중 평균 가격(VWAP)**을 계산하여 정확한 평단가를 제공합니다.

### **2) 기술 스택 & 핵심 개념 (TIL 연계)**

* **Language:** Python 3.9+
* **Library:** Pandas, NumPy
* **Key Concepts:**
* **OOP:** 전처리 로직을 `DataCleaner` 클래스로 캡슐화하여 재사용성 확보.
* **Pandas ETL:** `map`(고속 매핑), `to_datetime`, `astype` 활용.
* **Advanced GroupBy:** `transform('mean')`을 활용한 그룹별 결측치 채우기.



### **3) 데이터 흐름 (Pipeline)**

1. **Input:** `df_raw` (Dirty CSV 형태)
2. **Process (Class: DataCleaner):**
* `clean_date`: 날짜 파싱 및 오류 행 제거
* `clean_side`: 매수/매도 Boolean(`True`/`False`) 통일
* `clean_price`: 특수문자 제거 및 수치형 변환
* `fill_missing`: 종목별(ticker) 그룹 평균으로 가격 보간


3. **Output:** 종목별 가중 평균 가격(VWAP) 리포트


In [2]:
import pandas as pd
import numpy as np

# 실무에서 마주칠 법한 '더러운' 데이터 생성
data = {
    'date': ['2024-01-01', '2024/01/02', '2024.01.03', '2024-01-01', '2024-01-05', 'invalid_date'],
    'ticker': ['005930', '005930', '035420', '035420', '005930', '035420'], # 삼성전자, 네이버
    'side': ['Buy', 'B', 'sell', 'S', 'buy', 'Sell'], # 매수/매도 표기가 제각각
    'price': ['70000', '71,000', np.nan, '250000', None, '245000'], # 문자열, 결측치 혼재
    'qty': [10, 5, 3, 2, 10, 5]
}

df_raw = pd.DataFrame(data)

print("--- [Before] 더러운 원본 데이터 ---")
print(df_raw.info())
display(df_raw.head())

--- [Before] 더러운 원본 데이터 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    6 non-null      object
 1   ticker  6 non-null      object
 2   side    6 non-null      object
 3   price   4 non-null      object
 4   qty     6 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 372.0+ bytes
None


,date,ticker,side,price,qty
0,2024-01-01,005930,Buy,70000,10
1,2024/01/02,005930,B,"71,000",5
2,2024.01.03,035420,sell,NaN,3
3,2024-01-01,035420,S,250000,2
4,2024-01-05,005930,buy,None,10


In [3]:
df_raw.head(10)

,date,ticker,side,price,qty
0,2024-01-01,005930,Buy,70000,10
1,2024/01/02,005930,B,"71,000",5
2,2024.01.03,035420,sell,NaN,3
3,2024-01-01,035420,S,250000,2
4,2024-01-05,005930,buy,None,10
5,invalid_date,035420,Sell,245000,5


In [4]:
class DataCleaner:
    def __init__(self, df):
        self.df = df.copy() # 원본 훼손 방지
    
    def clean_date(self):
        print(">> 날짜 정규화 진행 중...")
        # [TODO 1] 'date' 컬럼을 datetime 객체로 변환하세요.
        # 힌트: pd.to_datetime() 사용. 
        # 변환 불가능한 값('invalid_date')은 에러 대신 NaT로 처리되도록(errors='coerce') 설정.
        self.df['date'] = pd.to_datetime(self.df["date"], errors='coerce', format='mixed')
        
        # [TODO 2] NaT(날짜 변환 실패)가 있는 행을 제거하세요.
        self.df = self.df.dropna(subset=['date'])
        return self

    def clean_side(self):
        print(">> 매수/매도 표준화 진행 중...")
        # [TODO 3] 'side' 컬럼의 문자열을 모두 소문자로 변환하고 양쪽 공백을 제거하세요.
        # 힌트: .str 접근자 사용
        side_clean = self.df["side"].astype(str).str.strip().str.lower()
        
        # [TODO 4] 아래 매핑 사전을 이용하여 데이터를 True/False로 변환하세요.
        # buy, b -> True / sell, s -> False
        # 힌트: apply 대신 map()을 사용해야 성능이 좋습니다.
        mapping = {'buy': True, 'b': True, 'sell': False, 's': False}
        self.df['side'] = side_clean.map(mapping)
        return self

    def clean_price(self):
        print(">> 가격 데이터 숫자 변환 진행 중...")
        # [TODO 5] 'price' 컬럼의 쉼표(,)를 제거하고 숫자(float)로 변환하세요.
        # 힌트 1: .astype(str)로 먼저 문자열화 -> .str.replace(',', '')
        # 힌트 2: pd.to_numeric()을 사용하거나 .astype(float) 사용
        self.df['price'] = (
            self.df['price'].astype(str)
            .str.replace(",","") # 쉼표 제거
        )
        self.df['price'] = pd.to_numeric(self.df['price'], errors='coerce')
        return self

    def fill_missing_price(self):
        print(">> 결측치 보간(Imputation) 진행 중...")
        # [TODO 6] 가격이 NaN인 곳을 '해당 종목(ticker)의 평균 가격'으로 채우세요.
        # 힌트: fillna() 내부에 df.groupby('...')['...'].transform('mean') 사용
        # 수업 핵심: transform을 쓰면 집계된 값이 원래 데이터 길이에 맞춰 확장됩니다.
        
        ticker_mean = self.df.groupby("ticker")["price"].transform('mean')
        self.df['price'] = self.df['price'].fillna(ticker_mean)
        return self

    def get_clean_data(self):
        # 파이프라인 실행 (메서드 체이닝)
        self.clean_date()
        self.clean_side()
        self.clean_price()
        self.fill_missing_price()
        return self.df

# --- 실행 및 검증 ---
cleaner = DataCleaner(df_raw)
df_clean = cleaner.get_clean_data()

print("\n--- [After] 정제 완료 데이터 ---")
display(df_clean)
print(df_clean.info())

>> 날짜 정규화 진행 중...
>> 매수/매도 표준화 진행 중...
>> 가격 데이터 숫자 변환 진행 중...
>> 결측치 보간(Imputation) 진행 중...

--- [After] 정제 완료 데이터 ---


,date,ticker,side,price,qty
0,2024-01-01,005930,True,70000.0,10
1,2024-01-02,005930,True,71000.0,5
2,2024-01-03,035420,False,250000.0,3
3,2024-01-01,035420,False,250000.0,2
4,2024-01-05,005930,True,70500.0,10


<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5 non-null      datetime64[ns]
 1   ticker  5 non-null      object        
 2   side    5 non-null      bool          
 3   price   5 non-null      float64       
 4   qty     5 non-null      int64         
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 205.0+ bytes
None


In [5]:
# [TODO 7] '거래대금(amount)' 파생 컬럼 생성 (가격 * 수량)
df_clean['amount'] = df_clean['price'] * df_clean['qty']

# [TODO 8] 종목(ticker)별로 'amount'와 'qty'의 합계(sum) 구하기
grouped = df_clean.groupby('ticker')[['amount', 'qty']].sum()

# [TODO 9] 가중 평균(vwap) 계산 (총 거래대금 / 총 거래량)
grouped['vwap'] = grouped['amount'] / grouped['qty']

grouped = grouped.reset_index()


print("\n--- [Result] 종목별 가중 평균가(VWAP) 리포트 ---")
display(grouped)


--- [Result] 종목별 가중 평균가(VWAP) 리포트 ---


,ticker,amount,qty,vwap
0,005930,1760000.0,25,70400.0
1,035420,1250000.0,5,250000.0


In [9]:
df_clean.to_csv("data/PJ05_Cleaned_Fintech_Data.csv", index=False, encoding='utf-8-sig')
grouped.to_csv("data/PJ05_VWAP_Fintech_Data.csv", index=False, encoding='utf-8-sig')

In [12]:
clean_df = pd.read_csv("data/PJ05_Cleaned_Fintech_Data.csv", dtype={'ticker': str})
clean_df

,date,ticker,side,price,qty,amount
0,2024-01-01,005930,True,70000.0,10,700000.0
1,2024-01-02,005930,True,71000.0,5,355000.0
2,2024-01-03,035420,False,250000.0,3,750000.0
3,2024-01-01,035420,False,250000.0,2,500000.0
4,2024-01-05,005930,True,70500.0,10,705000.0


In [13]:
VWAP_df = pd.read_csv("data/PJ05_VWAP_Fintech_Data.csv", dtype={'ticker': str})
VWAP_df

,ticker,amount,qty,vwap
0,005930,1760000.0,25,70400.0
1,035420,1250000.0,5,250000.0


## 회고
- 오늘은 짧은 분량으로 빠르게 프로젝트를 마쳤다.
- 그럼에도 EDA의 초반 학습 내용이었던 전처리 부분을 다시 복습한다는 느낌으로 가볍게 살펴볼 수 있어 좋았다.
- 내일 프로젝트부터는 조금 무게를 두고 좀 더 많은 개념을 담을 수 있도록 해야겠다.
    - 전처리 -> 파생 컬럼 생성 -> 매핑 테이블 병합 -> 그룹 집계/피벗 -> 결과 저장까지.
## 확장
- 각 종목 별의 수익률을 계산하는 로직도 추가할 수 있을 것이다.
- date 데이터를 사용해 누적된 거래 내역(history) 데이터를 활용하여 시계열 데이터로서 활용해볼 수 있을 것이다.